In [15]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import subprocess
import os
import sys
import pyrodigal
import pyrodigal_gv
import multiprocessing.pool
import math
import re
import time

In [31]:
single_fasta = "/Users/u241374/mike_tisza/sandbox/four_HMP_isolate_genomes1.fna"

prod_single_out = "/Users/u241374/mike_tisza/sandbox/single_out.prod.faa"

prod_gv_single_out = "/Users/u241374/mike_tisza/sandbox/single_out.prod_gv.faa"

prod_thread_out = "/Users/u241374/mike_tisza/sandbox/thread_out.prod.faa"


In [32]:
%%time

orf_finder = pyrodigal.GeneFinder(meta = True)


for record in SeqIO.parse(single_fasta, "fasta"):
    for i, pred in enumerate(orf_finder.find_genes(bytes(record.seq))):
        print(f">{record.id}_{i+1}", file = open(prod_single_out, "a"))
        print(pred.translate(), file = open(prod_single_out, "a"))

CPU times: user 35.2 s, sys: 1.35 s, total: 36.5 s
Wall time: 36.9 s


In [29]:
%%time

orf_finder = pyrodigal_gv.ViralGeneFinder(meta = True)


for record in SeqIO.parse(single_fasta, "fasta"):
    for i, pred in enumerate(orf_finder.find_genes(bytes(record.seq))):
        print(f">{record.id}_{i+1}", file = open(prod_gv_single_out, "a"))
        print(pred.translate(), file = open(prod_gv_single_out, "a"))

CPU times: user 36.6 s, sys: 6.69 s, total: 43.3 s
Wall time: 45.3 s


In [ ]:
%%time

records = SeqIO.parse(single_fasta, "fasta")

orf_finder = pyrodigal.GeneFinder(meta = True)

with multiprocessing.pool.ThreadPool() as pool:
    for record in records:
        predictions = pool.map(orf_finder.find_genes, str(record.seq))

In [40]:
%%time

records = SeqIO.parse(single_fasta, "fasta")

orf_finder = pyrodigal_gv.ViralGeneFinder(meta = True)

def _find_genes(record):
    genes = orf_finder.find_genes(str(record.seq))
    return (record.id, genes)

with multiprocessing.pool.ThreadPool() as pool:
    with open(prod_thread_out, "w") as dst:
        for record_id, genes in pool.imap(_find_genes, records):
            genes.write_translations(dst, sequence_id=record_id)
#        predictions = pool.map(orf_finder.find_genes, records.seq)
#        for record_id, genes in predictions:
#            genes.write_genes(dst, sequence_id=record.id)

CPU times: user 54.4 s, sys: 389 ms, total: 54.8 s
Wall time: 8.5 s


In [43]:
%%time

records = SeqIO.parse(single_fasta, "fasta")

orf_finder = pyrodigal.GeneFinder(meta = True)

def _find_genes(record):
    genes = orf_finder.find_genes(str(record.seq))
    return (record.id, genes)

with multiprocessing.pool.ThreadPool() as pool:
    with open(prod_thread_out, "w") as dst:
        for record_id, genes in pool.imap(_find_genes, records):
            genes.write_translations(dst, sequence_id=record_id)
#        predictions = pool.map(orf_finder.find_genes, records.seq)
#        for record_id, genes in predictions:
#            genes.write_genes(dst, sequence_id=record.id)

CPU times: user 50.1 s, sys: 276 ms, total: 50.4 s
Wall time: 7.9 s


In [ ]:
for record in SeqIO.parse(single_fasta, "fasta"):
    print(record)

In [39]:
print(SeqIO.parse(single_fasta, "fasta"))